In [1]:
import re
import json
import requests
from datetime import datetime, timedelta
import torch

In [2]:
api_key = 'c6dfc4d92a8f972d237ef696ec87b37a'

In [3]:
def get_weather(city):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
    response = requests.get(url)
    return response.json() if response.status_code == 200 else None

def get_forecast(city):
    url = f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={api_key}&units=metric"
    response = requests.get(url)
    return response.json() if response.status_code == 200 else None

In [4]:
def extract_intent_data(intent_response):
    json_pattern = re.compile(r'\{.*\}')
    json_match = json_pattern.search(intent_response)
    if json_match:
        intent_data_str = json_match.group()
        intent_data_str = intent_data_str.replace("'", "\"")
        try:
            return json.loads(intent_data_str)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")
    return {}

def fetch_weather_data(intent, city):
    if intent == "current_weather":
        return get_weather(city)
    elif intent == "forecast" or intent == 'forecast_weather':
        return get_forecast(city)
    return None

def parse_forecast(api_response, city):
    if not api_response:
        return "Sorry, I couldn't fetch the weather information."

    forecast_list = api_response['list']
    tomorrow_date = (datetime.utcnow() + timedelta(days=1)).strftime('%Y-%m-%d')
    filtered_forecasts = [forecast for forecast in forecast_list if tomorrow_date in forecast['dt_txt']]
    
    if filtered_forecasts:
        selected_forecast = filtered_forecasts[0]
        forecast_temp = selected_forecast['main']['temp']
        forecast_weather = selected_forecast['weather'][0]['description']
        wind_speed = selected_forecast['wind']['speed']
        humidity = selected_forecast['main']['humidity']

        return (
            f"The forecast for {city} on {tomorrow_date} is {forecast_weather} with a temperature of {forecast_temp}°C, "
            f"wind speed of {wind_speed} meters per second, and humidity of {humidity}%."
        )
    return f"Sorry, I couldn't fetch the weather information for {city} for tomorrow."

def generate_assistant_response(user_input, api_response, model, tokenizer):
    messages = [
        {"from": "human", "value": user_input},
        {"from": "api", "value": api_response}
    ]
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to("cuda")

    outputs = model.generate(input_ids=inputs, max_new_tokens=64, use_cache=True)
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

In [5]:
def inference_pipeline(user_input, model, tokenizer):
    # Step 1: Intent Identification
    messages = [{"from": "human", "value": user_input}]
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to("cuda")

    intent_output = model.generate(input_ids=inputs, max_new_tokens=64, use_cache=True)
    intent_response = tokenizer.batch_decode(intent_output, skip_special_tokens=True)[0]

    # Extract intent data
    intent_data = extract_intent_data(intent_response)
    intent = intent_data.get("intent")
    city = intent_data.get("entities", {}).get("city")
    date = intent_data.get("entities", {}).get("date")

    # Step 2: Fetch weather data from API
    api_response = fetch_weather_data(intent, city)
    parsed_response = parse_forecast(api_response, city)

    # Step 3: Generate the final assistant response using LLM
    final_response = generate_assistant_response(user_input, parsed_response, model, tokenizer)
    return final_response